## Valós időben történő kiértékelés (azaz majdnem valós időben).

##### A micro-controller-en futó program socket serverként üzemel. A rá csatlakozó kliens felé 5 másodpercenként (azaz 200 minta összegyűjtése után) elküldi az IMU-ról gyűjtött adatokat. Cél az azonnali kiértékelés majd várakozás a következő csomagra.
A program futtatása előtt csatlakozni kell az ESP32_KETTLEBELL wifi hálózathoz!

In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
import socket
import pandas as pd
import numpy as np
from ipycanvas import Canvas, hold_canvas
from threading import Timer


In [2]:
#process_imu_convolution által mentett modell betöltése
model = load_model('kettlebell_model.h5')
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200, 6)]          0         
                                                                 
 conv1d (Conv1D)             (None, 198, 72)           1368      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 99, 72)           0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 99, 72)            0         
                                                                 
 conv1d_1 (Conv1D)           (None, 97, 18)            3906      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 48, 18)           0         
 1D)                                                         

In [3]:
canvas = Canvas(width=1000, height=100)
canvas.font = '48px serif'
text = ''
alpha = 1.0
timer=Timer(0.2, lambda:None)
def timer_cb(): 
    global text, alpha, timer
    with hold_canvas(canvas):
        canvas.clear()
        canvas.global_alpha = alpha
        canvas.fill_style = 'black'
        canvas.fill_text(text, 0, 40)
        alpha = alpha - 0.02
        if alpha > 0.1:
            timer=Timer(0.1, timer_cb)
            timer.start()
def print_really_big(txt):
    global text, alpha, timer
    text=txt
    alpha = 1.0
    timer.cancel()
    timer=Timer(0.1, timer_cb)
    timer.start()


In [4]:
#kapcsolat felépítése a microcontroller felé (sikeres csatlakozáskor rövid ideig a kijelzőn:"OK")
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(('192.168.4.1',8000))

In [5]:
#Gyűjtött adatok olvasása, kiértékelése
display(canvas)
while True:
    #milisec: 4 bytes + 6 csatorna * 2 bytes = 16 bytes * 200 = 3200
    data = s.recv(3200) 
    rows = []
    for j in range(200):
        offset = j*8*2    
        b=data[offset:offset+16]
        time_ms = int.from_bytes(b[0:4], byteorder='big', signed=True)
        imus = [time_ms]
        for i in range(6):
            imus.append(int.from_bytes(b[i*2+4:i*2+6], byteorder='big', signed=True))
        rows.append(imus)
    df = pd.DataFrame(rows,columns=['time_ms','acc_x','acc_y','acc_z','gyr_x','gyr_y','gyr_z'])
    X = np.array([])
    #osztályelnevezések
    target_dict={0:'DEADLIFT',1:'HALO',2:'CLEAN', 3:'SNATCH',4:'SWING',5:'NOTHING'}
    #lebegőpontos átalakítás a model alkalmazása előtt
    X=(df.loc[:,'acc_x':'gyr_z'].to_numpy()+32768)/65536
    X=X.reshape(1,200,6)
    pred = model.predict(X)
    #print(target_dict[np.argmax(pred)], np.max(pred), end='           \r')
    if np.max(pred) > 0.95:
        print_really_big(target_dict[np.argmax(pred)]+' '+str(np.max(pred)))
    else:
        print_really_big('Sorry, Not sure!')

Canvas(height=100, width=1000)

ConnectionAbortedError: [WinError 10053] A felépített kapcsolatot az állomás szoftvere megszakította